# Amplitude regression

We want to train a network to predict the amplitude for the process $gg\rightarrow\gamma\gamma g$.  So the amplitude depends on the 4-momentum of 5 particles: 2 incoming gluons, 2 outgoing photons, and one outgoing gluon.  

The incoming gluons will have no transverse momentum, but their total momentum along the beam pipe is not necessarily zero.

The network we will train is a simple fully connected dense network.  This means that the input and output can only be vectors of real numbers.  We want to input the kinematic information on the particles to the network, and train the network to output the corresponding amplitude.  So we need to think about how we input our kinematic 4-vectors to the network, this generally depends on the network architecture we are using.

#### Outline / tasks:
 - Imports \& plotting set-up
 - Loading the data
 - Visualising the data
     - visualise some of the kinematics of the process (transverse momentum of photons/gluons, MET)
     - histogram the amplitudes
 - Preprocessing the data
     - neural networks like $\mathcal{O}(1)$ numbers
     - how should we preprocess the data?
 - Datasets and dataloaders
     - details are in the tensorflow docs
 - Building the neural network
     - construct a simple neural network with 2 hidden layers with some number of nodes, it's up to you
     - tip: use linear layers followed by relu activations
     - all the relevant details are in the tensorflow docs
 - Plot the train and validation losses as a function of the epochs
     - to check the the network is training we should plot the train and validation losses as a function of the epochs
 - Study the results
     - beyond the losses, how can we check that our network is making sensible predictions?
 - Can you think of ways to improve your network or your training?  List them!

### Imports

In [ ]:
import os
import sys
import random
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, activations

#### Plotting set-up

In [ ]:
from matplotlib import pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings("ignore")
from matplotlib.lines import Line2D
from matplotlib.font_manager import FontProperties
import matplotlib.colors as mcolors
import colorsys

labelfont = FontProperties()
labelfont.set_family('serif')
labelfont.set_name('Times New Roman')
labelfont.set_size(14)

axislabelfont = FontProperties()
axislabelfont.set_family('serif')
axislabelfont.set_name('Times New Roman')
axislabelfont.set_size(22)

tickfont = FontProperties()
tickfont.set_family('serif')
tickfont.set_name('Times New Roman')
tickfont.set_size(16)

axisfontsize = 16
labelfontsize = 16

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["mathtext.default"] = "rm"
plt.rcParams['text.usetex'] = True

## Loading the data

In [ ]:
trn_dat = np.load( "tutorial-2-data/trn_dat.npy" )
trn_amp = np.load( "tutorial-2-data/trn_amp.npy" )

val_dat = np.load( "tutorial-2-data/val_dat.npy" )
val_amp = np.load( "tutorial-2-data/val_amp.npy" )

tst_dat = np.load( "tutorial-2-data/tst_dat.npy" )
tst_amp = np.load( "tutorial-2-data/tst_amp.npy" )

In [ ]:
print( f"train data shape: {trn_dat.shape}" )
print( f"train amp  shape: {trn_amp.shape}" )
print( f"test  data shape: {tst_dat.shape}" )
print( f"test  amp  shape: {tst_amp.shape}" )
print( f"val   data shape: {val_dat.shape}" )
print( f"val   amp  shape: {val_amp.shape}" )

The data is organised as follows:
 - the shape corresponds to ( number of events, number of particles, 4-momentum )
 - the particles are arranged as follows
     - the first two entries are the two incoming gluons
     - the next two particles are the outgoing photons
     - the last particle is the outgoing gluon
 - the incoming gluons and incoming photons are arranged by transverse memontum $p_T$

## Visualising the data

Below we will make some kinematic plots of the events in the training sample.  Note however that these are not the physical distributions we would measure at the LHC!  In our training data each of these events is associated with an amplitude, which tells us the probability that the event will be produced in the gluon-gluon interaction.  So to get the physical distributions these events would need to be 'weighted' by their amplitude.  However, right now we just want to visualise our training dataset to see what preprocessing we should do.

In [ ]:
def get_init_pz( ev ):
    return ev[0][3] + ev[1][3]

def get_mass( fv ):
    msq = np.round( fv[0]**2 - fv[1]**2 - fv[2]**2 - fv[3]**2 , 5 )
    if msq>0:
        return np.sqrt( msq )
    elif msq<0:
        raise Exception( "mass squared is less than zero" ) 
    else:
        return 0
    
def get_pt( fv ):
    ptsq = np.round( fv[1]**2 + fv[2]**2 , 5 )
    if ptsq>0:
        return np.sqrt( ptsq )
    elif ptsq<0:
        raise Exception( "$p_T$ squared is less than zero" ) 
    else:
        return 0
    
def get_met( ev ):
    return np.abs( np.sum( [ fv[1]+fv[2] for fv in ev ] ) )

Plotting the initial $p_z$ of the events in the training sample.

In [ ]:
#....#

We can plot a histogram of the amplitudes for the training data.

In [ ]:
#...#

The amplitudes span about 4 orders of magnitude...

Plotting the leading photon $p_T$.

In [ ]:
#...#

Plot the final state gluon $p_T$.

In [ ]:
#...#

The mean final state gluon $p_T$ in GeV is:

In [ ]:
#...#

Plotting the missing transverse energy (MET) for each event.

In [ ]:
#...#

## Preprocessing the data

We will be using a dense network, so the data needs to be in vector format.  We will collapse the data for each event to a single vector of dimension $5\times4=20$.  The fact that the data is ordered here is important.  To predict the amplitude given the kinematics, the network needs to know which entries correspond to which particles in the process.

In [ ]:
nev = trn_dat.shape[0]
trn_datf = np.reshape( trn_dat, (nev,-1) )
val_datf = np.reshape( val_dat, (nev,-1) )
tst_datf = np.reshape( tst_dat, (nev,-1) )

Check the shape:

In [ ]:
trn_datf.shape

There are further preprocessing steps we can take.  For example, the inputs are numerically very large $\mathcal{O}(100)$ and span a large range.  So we could re-scale the inputs by a constant number, or even take the logarithm of the inputs.

In [ ]:
trn_datf = #....#
val_datf = #....#
tst_datf = #....#

We should also preprocess the amplitude data.  As we seen in a plot above, the amplitudes span about 4 orders of magnitude.  This could be difficult for the network to interpolate.

In [ ]:
trn_ampl = #....#
val_ampl = #....#
tst_ampl = #....#

The new distribution looks nicer:

In [ ]:
#....#

## Building the neural network

We'll build a simple network with one input and one output layer, and two hidden layers.  We define the dimensions of these layers below.

In [ ]:
ipt_dim = #...#
opt_dim = 1
#...

In [ ]:
def make_model(ipt_dim, opt_dim, #....#):
    input_la = keras.Input(shape=(ipt_dim,)) #input layer
    #....#
    model.add(layers.Dense(opt_dim)) #output layer
    return model

We need to decide what function we want the neural network to optimise, i.e. the loss function.  There are a number of choices to decide from, the key point is that the loss function should be minimised when the neural network correctly predicts the amplitude given the kinematical information on the event.

In [ ]:
model = make_model(ipt_dim, opt_dim, #....#)
loss_mse =  #loss function
opt_SGD =  #optimizer
model.compile(loss=loss_mse, optimizer = opt_SGD)
model.summary()

In [ ]:
hist = model.fit(#....#)

## Plot the train and validation losses as a function of the epochs

In [ ]:
#.....#

We haven't used the test data here.  The test data becomes useful if our neural network overfits the training data, and we want to stop training the network early.  In this case we use the validation data to decide at which epoch to stop training, based on some early stopping condition.  For example, if the validation loss does not improve for 10 epochs, we might decide to stop training.  If we have saved the network at certain epochs during training, we would then select the network with the smallest validation loss to use.  The test data is then the final dataset we test on, since it was not used during training or in the early stopping procedure.

## Study the results

Now we want to get some visualisation of how well our amplitude regression has worked.

The simplest thing we can do is to pass our data through the neural network to get a predicted amplitude for each event, then histogram this and compare it to the histogram of the true amplitudes.

In [ ]:
pred_trn_ampls = #....#
pred_val_ampls = #....#
pred_test_ampls = #....#

First for the training data:

In [ ]:
#....#

In [ ]:
matplotlib.rcParams["text.usetex"]  = True

We can also try to visualise the error on the predicted amplitudes by plotting $\text{abs}\left((A_{\text{true}}-A_{\text{pred}})/A_{\text{true}} \right)$.

In [ ]:
#...#

And then for the validation data:

In [ ]:
#...#

In [ ]:
#....#

And for the test data:

In [ ]:
#....#